# Analyze Simulation Results
This tutorial will use the simulation results from the [finite amplitudes stimulation](1_finite_amp.ipynb) and [activation threshold search](2_activation_threshold.ipynb) tutorials. We will analyze the response in transmembrane electric potential (Vm) and gating variables to extracellular stimulation of a fiber over space and time.

## Create the fiber and set up simulation
As before, we create fiber, waveform, potentials and stimulation object.

In [ ]:
import numpy as np
from pyfibers import build_fiber, FiberModel, ScaledStim

# create fiber model
n_sections = 265
fiber = build_fiber(FiberModel.MRG_INTERPOLATION, diameter=10, n_sections=n_sections)
print(fiber)

# Setup for simulation. Add zeros at the beginning so we get some baseline for visualization
waveform = np.concatenate((np.zeros(100), np.ones(100), np.zeros(48000)))  # biphasic rectangular pulse

fiber.potentials = fiber.point_source_potentials(0, 250, fiber.length / 2, 1, 10)

time_step = 0.001  # TODO increase timestep and shorten simulation time in all notebooks
time_stop = 50

# Create stimulation object
stimulation = ScaledStim(waveform=waveform, dt=time_step, tstop=time_stop)

## Run search for activation threshold

As before, we can simulate the response to a single stimulation pulse.

In [ ]:
stimamp = -1.5  # mA
ap, time = stimulation.run_sim(stimamp, fiber)
print(f'Number of action potentials detected: {ap}')
print(f'Time of last action potential detection: {time} ms')

Before running the simulation, we did not tell the fiber to save any data. Therefore, no transmembrane potential (Vm) or gating variable information was stored. We can confirm this using Python's hasattr() command.

In [ ]:
# checks if the fiber object has the given attribute:
# transmembrane_potentials (vm), gating variables (gating) and transmembrane currents (im)
saved_vm = fiber.vm is not None
print(f"Saved Vm?\n\t{saved_vm}\n")

saved_gating = fiber.gating is not None
print(f"Saved gating?\n\t{saved_gating}")

saved_im = fiber.im is not None
print(f"Saved Im?\n\t{saved_im}")

Let's control the fiber to save the membrane voltage and gating variables and then re-run the simulation.

In [ ]:
fiber.set_save_vm()  # save membrane voltage
fiber.set_save_gating()  # save gating variables
fiber.set_save_im()  # save membrane current
ap, time = stimulation.run_sim(-1.5, fiber)

Now that we have saved membrane voltage and gating variables, let's take a look at them.

In [ ]:
print(fiber.vm)
print(fiber.gating)
print(fiber.im)

We have a neuron `Vector` object for each node of the fiber.

NOTICE: By default MRG fibers are created with passive end nodes (see that the first and last values are "None") to prevent initiation of action potentials at the terminals due to edge-effects. We are simulating the response of a fiber of finite length local to the site of stimulation.

Next, let's plot the transmembrane voltage for one end compartment and the center compartment to visualize the fiber response to stimulation.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

end_node = 1  # not zero since it was passive and therefore has no data to show!
center_node = int(np.floor(0.5 * (1 + (n_sections - 1) / 11)))

sns.set(font_scale=1.5, style='whitegrid', palette='colorblind')

plt.figure()
plt.plot(
    np.array(stimulation.time)[:2000], list(fiber.vm[end_node])[:2000], label='end node', color='royalblue', linewidth=2
)
plt.plot(
    np.array(stimulation.time)[:2000],
    list(fiber.vm[center_node])[:2000],
    label='center node',
    color='mediumturquoise',
    linewidth=2,
)
plt.legend()
plt.xlabel('Time (ms)')
plt.ylabel('$V_m$ $(mV)$')
ax2 = plt.gca().twinx()
ax2.plot(np.array(stimulation.time)[:2000], stimamp * waveform[:2000], 'k--', label='Stimulus')
ax2.legend(loc=4)
ax2.grid(False)
plt.ylabel('Stimulation amplitude (mA)')
plt.show()

We can plot a heatmap of the voltage across all compartments over time.

In [ ]:
import pandas as pd

data = pd.DataFrame(np.array(fiber.vm[1:-1]))
vrest = fiber[0].e_pas
print('Membrane rest voltage:', vrest)
g = sns.heatmap(
    data,
    cbar_kws={'label': '$V_m$ $(mV)$'},
    cmap='seismic',
    vmax=np.amax(data.values) + vrest,
    vmin=-np.amax(data.values) + vrest,
)
plt.xlim([0, 1000])
plt.ylabel('Node index')
plt.xlabel('Time (ms)')
tick_locs = np.linspace(0, len(np.array(stimulation.time)[:1000]), 9)
labels = [round(np.array(stimulation.time)[int(ind)], 2) for ind in tick_locs]
g.set_xticks(ticks=tick_locs, labels=labels)
plt.title(
    'Membrane voltage over time\
          \nRed=depolarized, Blue=hyperpolarized'
)

Running a threshold search will also save our variables. Let's try plotting Vm at threshold.

In [ ]:
amp, ap = stimulation.find_threshold(fiber)
print(f'Activation threshold: {amp} mA')

In [ ]:
# plot vm
plt.figure()
plt.plot(
    np.array(stimulation.time)[:2000], list(fiber.vm[end_node])[:2000], label='end node', color='royalblue', linewidth=2
)
plt.plot(
    np.array(stimulation.time)[:2000],
    list(fiber.vm[center_node])[:2000],
    label='center node',
    color='mediumturquoise',
    linewidth=2,
)
plt.legend()
plt.xlabel('Time (ms)')
plt.ylabel('$V_m$ $(mV)$')
ax2 = plt.gca().twinx()
ax2.plot(np.array(stimulation.time)[:2000], amp * waveform[:2000], 'k--', label='Stimulus')
ax2.legend(loc=4)
ax2.grid(False)
plt.ylabel('Stimulation amplitude (mA)')
plt.show()

In [ ]:
# plot heatmap
data = pd.DataFrame(np.array(fiber.vm[1:-1]))
vrest = fiber[0].e_pas
print('Membrane rest voltage:', vrest)
g = sns.heatmap(
    data,
    cbar_kws={'label': '$V_m$ $(mV)$'},
    cmap='seismic',
    vmax=np.amax(data.values) + vrest,
    vmin=-np.amax(data.values) + vrest,
)
plt.xlim([0, 1000])
tick_locs = np.linspace(0, len(np.array(stimulation.time)[:1000]), 9)
labels = [round(np.array(stimulation.time)[int(ind)], 2) for ind in tick_locs]
g.set_xticks(ticks=tick_locs, labels=labels)
plt.ylabel('Node index')
plt.xlabel('Time (ms)')
plt.title(
    'Membrane voltage over time. \
          \nRed=depolarized, Blue=hyperpolarized'
)

Let's take a look at the gating variables.

In [ ]:
# plot gating variables
plt.figure()
for var in fiber.gating:
    plt.plot(np.array(stimulation.time)[:2000], list(fiber.gating[var][6])[:2000], label=var)
plt.legend()
plt.xlabel('Time (ms)')
plt.ylabel('Gating probability')
ax2 = plt.gca().twinx()
ax2.plot(np.array(stimulation.time)[:2000], amp * waveform[:2000], 'k--', label='Stimulus')
ax2.legend(loc=4)
ax2.grid(False)
plt.ylabel('Stimulation amplitude (mA)')
plt.show()

...and the transmembrane currents. 

In [ ]:
plt.figure()
fig, axs = plt.subplots(3, 1, figsize=(5, 5), sharex=True, gridspec_kw={'hspace': 0.3})
plt.sca(axs[0])
# plot stimulus
plt.plot(np.array(stimulation.time)[:2000], amp * waveform[:2000], 'k--', label='Stimulus')
plt.title('Stimulus')
plt.sca(axs[1])
# plot membrane voltage
plt.plot(
    np.array(stimulation.time)[:2000],
    list(fiber.vm[center_node])[:2000],
    color='mediumturquoise',
    linewidth=2,
    label='$V_m$',
)
# plot im
plt.plot(
    np.array(stimulation.time)[:2000],
    list(fiber.im[center_node])[:2000],
    color='mediumturquoise',
    linewidth=2,
    label='$I_m$',
    ls='--',
)
plt.title('Center node')
plt.legend()
plt.sca(axs[2])
# plot end node
plt.plot(
    np.array(stimulation.time)[:2000], list(fiber.vm[end_node])[:2000], color='royalblue', linewidth=2, label='$V_m$'
)
plt.plot(
    np.array(stimulation.time)[:2000],
    list(fiber.im[end_node])[:2000],
    color='royalblue',
    linewidth=2,
    label='$I_m$',
    ls='--',
)
plt.title('End node')
plt.legend()
plt.xlim([0, 2])
axs[2].set_xlabel('Time (ms)')
plt.show()

Finally, we can use the data to make videos, which can help visualize how the fiber variables change over time.

In [ ]:
from moviepy.editor import VideoClip
from moviepy.video.io.bindings import mplfig_to_npimage

# %% make video of vm
fps = 30
skip = 10  # do every 10th timestep
stop = 2 / time_step  # stop after 2 milliseconds

duration = stop / fps / skip  # milliseconds
ylim = (np.amin(list(fiber.vm[1:-1]))), np.amax(list(fiber.vm[1:-1]))

fig, ax = plt.subplots()


def make_frame(i):
    """Create frame of video.

    :param i: index of frame, given to function by moviepy
    :returns: figure as image
    """
    ind = int(i * skip * fps)
    ax.clear()
    ax.set_ylim(ylim)
    ax.plot([v[ind] for v in fiber.vm[1:-1]], lw=3)
    plt.title(f'Time: {stimulation.time[ind]:0.1f} ms')
    plt.ylabel('$V_m$')
    plt.xlabel('Node index')
    plt.tight_layout()
    return mplfig_to_npimage(fig)


animation = VideoClip(make_frame, duration=duration)
animation.write_videofile('ap.mp4', fps=fps)
plt.close()

In [ ]:
# display video
from IPython.display import HTML

HTML(
    """
<video width="320" height="240" controls>
  <source src="ap.mp4" type="video/mp4">
</video>
"""
)

You can use the same technique to plot other data, such as transmembrane current or gating variables!